# Julia avanzado

## Metaprogramming

Julia, igual que Lisp, representa al código como una estructura de datos en el propio lenguaje. La posibilidad de escribir código que genere y modifique código es lo que se entiende por "Metaprogramming".

Ilustraremos algunos conceptos siguiendo el [manual](http://docs.julialang.org/en/release-0.4/manual/metaprogramming/). Cualquier línea de código, inicialmente es una cadena:

In [1]:
one_plus_one = "1+1"

"1+1"

El siguiente paso es convertir esta cadena en una *expresión*:

In [2]:
ex1 = parse(one_plus_one)

:(1 + 1)

In [3]:
typeof(ex1)

Expr

In [4]:
?Expr

search: 

No documentation found.

**Summary:**

```julia
type Expr <: Any
```

**Fields:**

```julia
head :: Symbol
args :: Array{Any,1}
typ  :: Any
```


Expr export nextprod expanduser expm exp2 exp expm1 exp10 expand



Claramente, un objeto tipo `Expr` tiene tres partes:

- Un `Symbol`, que se guarda en `head`:

In [5]:
ex1.head

:call

- Un vector (tipo `Any`) que contiene a los argumentos:

In [6]:
ex1.args

3-element Array{Any,1}:
  :+
 1  
 1  

- el tipo resultante de la expresión, que puede ser anotado por el usuario

In [7]:
ex1.typ

Any

Las expresiones también pueden ser escritas directamente:

In [8]:
ex2 = Expr(:call, :+, 1, 1)

:(1 + 1)

El punto importante es que el código en Julia está representado internamente por expresiones escritas en Julia.

La función `dump()` da información sobre la expresión:

In [9]:
dump(ex2)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1
  typ: Any


Expresiones más complicadas pueden ser construidas:

In [10]:
ex3 = parse("(4 + 4) / 2")

:((4 + 4) / 2)

In [11]:
Meta.show_sexpr(ex3)

(:call, :/, (:call, :+, 4, 4), 2)

Uno de los usos de `:` es crear símbolos; uno también puede usar `symbol()`:

In [12]:
:foo == symbol("foo")

true

In [ ]:
symbol(:var,'_',"sym")

Otro uso de `:` es crear expresiones sin usar `Expr`; a esto se le llama *quoting*.

In [13]:
ex = :(a+b*c+1)

:(a + b * c + 1)

In [19]:
dump(ex)

Expr 
  head: Symbol block
  args: Array(Any,(6,))
    1: LineNumberNode 
      file: Symbol In[15]
      line: Int64 4
    2: Expr 
      head: Symbol =
      args: Array(Any,(2,))
        1: Symbol x
        2: Int64 1
      typ: Any
    3: LineNumberNode 
      file: Symbol In[15]
      line: Int64 5
    4: Float64 2.1
    5: LineNumberNode 
      file: Symbol In[15]
      line: Int64 6
    6: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol +
        2: Symbol x
        3: Symbol y
      typ: Any
  typ: Any


Lo interesante, es que uno puede cambiar *programáticamente* el contenido de una expresión. Por ejemplo, en lugar de $ex=a + b*c + 1$ quiero tener $ex=a + b*c + 2.1$:

In [18]:
ex.args[4] = 2.1

2.1

In [20]:
dump(ex)

Expr 
  head: Symbol block
  args: Array(Any,(6,))
    1: LineNumberNode 
      file: Symbol In[15]
      line: Int64 4
    2: Expr 
      head: Symbol =
      args: Array(Any,(2,))
        1: Symbol x
        2: Int64 1
      typ: Any
    3: LineNumberNode 
      file: Symbol In[15]
      line: Int64 5
    4: Float64 2.1
    5: LineNumberNode 
      file: Symbol In[15]
      line: Int64 6
    6: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol +
        2: Symbol x
        3: Symbol y
      typ: Any
  typ: Any


Otra alternativa para definir expresiones complejas es usando el comando `quote`

In [15]:
# Lo siguiente es equivalente a:
# ex = :(x=1; y=2; x+y)
ex = quote
    x = 1
    y = 2
    x + y
end

quote  # In[15], line 4:
    x = 1 # In[15], line 5:
    y = 2 # In[15], line 6:
    x + y
end

In [16]:
dump(ex)

Expr 
  head: Symbol block
  args: Array(Any,(6,))
    1: LineNumberNode 
      file: Symbol In[15]
      line: Int64 4
    2: Expr 
      head: Symbol =
      args: Array(Any,(2,))
        1: Symbol x
        2: Int64 1
      typ: Any
    3: LineNumberNode 
      file: Symbol In[15]
      line: Int64 5
    4: Expr 
      head: Symbol =
      args: Array(Any,(2,))
        1: Symbol y
        2: Int64 2
      typ: Any
    5: LineNumberNode 
      file: Symbol In[15]
      line: Int64 6
    6: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol +
        2: Symbol x
        3: Symbol y
      typ: Any
  typ: Any



Para evaluar una expresión, usamos `eval()`. Respecto a esto, vale la pena notar que, mientras no se ha evaluado una expresión las variables internas no tienen ningún valor; sin embargo, una vez evaluada, los valores de las variables internas quedan definidas. Esto puede causar problemas, llamados *side effects*.

In [17]:
x

LoadError: LoadError: UndefVarError: x not defined
while loading In[17], in expression starting on line 1

In [21]:
y = 3

3

In [22]:
eval(ex)

4

In [23]:
x

1

In [24]:
y # y queda sobreescrita al evaluar la expresión !!

3

Uno incluso puede definir el valor de ciertas variables y usarlo como tal, o sustituirlo directamente:

In [25]:
z = 4

4

In [26]:
:(2*x+z)

:(2x + z)

In [27]:
eval(ans)

6

In [28]:
:(2*x+$z)

:(2x + 4)

In [29]:
:(2*x+$zz)

LoadError: LoadError: UndefVarError: zz not defined
while loading In[29], in expression starting on line 1

El punto es que uno puede definir funciones, que regresen expresiones, que pueden ser evaluadas

Un ejemplo más interesante es implementar los polinomios de Wilkinson, que se definen como:

\begin{equation}
    W_n(x) = \prod_{i=1}^n (x-i) = (x-1) (x-2) \cdots (x-n).
\end{equation}

Ciertamente este ejemplo se puede implementar a través de una recursión, pero es un buen ejemplo.

In [30]:
nombre(n::Int) = symbol( string("W_", n) )

nombre (generic function with 1 method)

In [31]:
nombre(3)

:W_3

In [32]:
function wilkinson(n::Int)
    # Checo que `n` sea >= 1
    @assert n >= 1
    
    ex = :(x-1)
    for i = 2:n
        ex = :( ($ex) * ( x-$i) )
    end
    ex_ret = :( $(nombre(n))(x) = $ex )
    ex_ret
end

wilkinson (generic function with 1 method)

In [33]:
wilkinson(0)

LoadError: LoadError: AssertionError: n >= 1
while loading In[33], in expression starting on line 1

In [34]:
wilkinson(3)

:(W_3(x) = begin  # In[32], line 9:
            ((x - 1) * (x - 2)) * (x - 3)
        end)

In [35]:
eval(ans)

W_3 (generic function with 1 method)

In [36]:
W_3(2)

0

In [37]:
eval(wilkinson(4))

W_4 (generic function with 1 method)

In [38]:
W_4(2.5)

0.5625

**Ejercicio 7:** Implementen el factorial con metaprogramming.

## Macros

En varias ocasiones hemos usado ya algunas instrucciones que incluyen `@` antes, por ejemplo, `@time`. Estos son los macros: Los macros son funciones cuyas entradas son *expresiones*, que son manipuladas y al final se evalúan.

In [ ]:
macro simple_example(expr)
    @show expr   # this is another macro !
    return 0     # for simplicity
end

In [ ]:
@simple_example(x+y)

In [ ]:
@simple_example a = x + y

Cambiemos un poco el macro `@simple_example`

In [ ]:
macro simple_example(expr)
    @show expr   # this is another macro !
    expr         # returns the input expression
end

In [ ]:
@simple_example x + y

In [ ]:
x = 3; y = 7;
@simple_example x + y

La gran sutileza de los macros es que, a diferencia de las funciones genéricas, permiten introducir y modificar código *antes* de que sea ejecutado, dado que los macros son ejecutados cuando el código se pasa (*parse time*).

El siguiente ejemplo está tomado del manual.

In [ ]:
macro twostep(arg)
    println("I execute at parse time. The argument is: ", arg)

    return :(println("I execute at runtime. The argument is: ", $arg))
end

In [ ]:
ex = macroexpand( :(@twostep :(1, 2, 3)) );

In [ ]:
macroexpand( (:@simple_example x+y) )

In [ ]:
typeof(ex)

In [ ]:
ex

In [ ]:
eval(ex)

In [ ]:
?@time

## Generación de código

Arriba, con la definición `wilkinson()` de hecho definimos una función que genera código. Si bien ése no es el mejor ejemplo para esto, el punto es que a veces, excepto por pequeños cambios (por ejemplo, en los operadores), el código esencialmente se repite.

Un ejemplo de lo anterior es el siguiente:

In [ ]:
type Vector2D{T<:Real}
    x :: T
    y :: T
end

In [ ]:
Vector2D(a,b) = Vector2D(promote(a,b)...)

In [ ]:
import Base: +, -

for fn = (:+, :-)
    println(fn)
    ex = quote
        function ($fn)(a::Vector2D, b::Vector2D)
            xx = ($fn)(a.x, b.x)
            yy = ($fn)(a.y, b.y)
            return Vector2D(xx, yy)
        end
    end
    println(ex)
    @eval $ex
end

In [ ]:
Vector2D(1, 2.2) - Vector2D(1,1)